In [2]:
# ! pip install librosa
# and such, in case it is needed...

In [3]:
import numpy as np
import os
import warnings
import librosa
import scipy
from scipy.signal import stft
from scipy.fftpack import fft
from scipy.fftpack import dct
import matplotlib.pyplot as plt
import math 

In [4]:
SR = 22050
MONO = True
NUM_MAT = 9
N_SONGS = 900
DEBUG = False

In [5]:
warnings.filterwarnings("ignore")

In [6]:
def import_csv(filename:str) -> np.ndarray:
    ans = np.genfromtxt(filename, skip_header = 1, delimiter=',')
    return ans [:,1:-1] # returns all feture values but the name and the quadrant


def export_csv(filename:str,  array:np.ndarray, fmt='%.18e') -> None:
    np.savetxt(filename, array, delimiter =',', fmt=fmt)



In [7]:
def print_debug(*args, **kwargs):
    if DEBUG:
        print(*args, **kwargs)

In [8]:

def normalize_features(matrix: np.ndarray) -> np.ndarray:
    maxs = np.max(matrix, axis = 0)
    mins = np.min(matrix, axis = 0)
    ans= np.array(matrix)
    ans[np.arange(len(matrix)),:] = (ans[np.arange(len(matrix)),: ] - mins)/(maxs-mins)
    return ans



In [9]:
def save_normalized_features() -> np.ndarray:
    data = import_csv("dataset/top100_features.csv")
    n_data = normalize_features(data)
    print_debug(n_data)
    # print_debug('shape: ', n_data.shape)
    export_csv('dataset/normalized_features.csv', n_data)
    return n_data

In [10]:
def ex2_1():
    save_normalized_features()

In [11]:
def zcr(sample, frame_length=2048,  hop_length=512):

    sample_size= len(sample)
    n_windows = (sample_size)// hop_length + 1
    zrc= np.zeros((1, n_windows))


    begs= np.arange(n_windows)*hop_length - frame_length//2
    begs[begs<0]=0
    ends= np.arange(n_windows)*hop_length + frame_length//2
    ends[ends>sample_size]= sample_size
    for i in range(n_windows):
        
        zrc[0,i] = np.sum(np.abs(np.diff(sample[begs[i]:ends[i]] > 0)))/(frame_length)


    return zrc

In [12]:
def rms(sample, frame_length=2048,  hop_length=512):

    sample_size= len(sample)
    n_windows = (sample_size)// hop_length + 1
    arr= np.zeros((1, n_windows))


    begs= np.arange(n_windows)*hop_length - frame_length//2
    begs[begs<0]=0
    ends= np.arange(n_windows)*hop_length + frame_length//2
    ends[ends>sample_size]= sample_size
    for i in range(n_windows):
        
        arr[0,i] =np.sqrt( np.sum(sample[begs[i]:ends[i]]**2)/(frame_length) )


    return arr

In [13]:
def spectral_rolloff(sample, sr= 22050, frame_length= 2048, hop_length=512):
    sample_size= len(sample)
    n_windows = (sample_size)// hop_length + 1
    arr= np.zeros((1, n_windows))


    begs= np.arange(n_windows)*hop_length - frame_length//2
    begs[begs<0]=0
    ends= np.arange(n_windows)*hop_length + frame_length//2
    ends[ends>sample_size]= sample_size


    for i in range(n_windows):

        x = sample[begs[i]:ends[i]]

        X = fft(x * np.hanning(len(x)))

        
        N = len(x)
        B = math.ceil(N/2)
        df = sr/N
        magnitude = np.abs(X[0:B])
        freq_bins = np.arange(B) * df

        total_mag = np.sum(magnitude)
        threshold = 0.85 * total_mag
        mag_per_index = np.cumsum(magnitude)
            
        aux = mag_per_index > threshold

        pos = np.where(aux.any(), aux.argmax(), B-1)


        arr[0,i] = freq_bins[pos]

    return arr  

In [14]:
def spectral_centroid(sample, sr= 22050, frame_length= 2048, hop_length=512):

    sample_size= len(sample)
    n_windows = (sample_size)// hop_length + 1
    arr= np.zeros((1, n_windows))


    begs= np.arange(n_windows)*hop_length - frame_length//2
    begs[begs<0]=0
    ends= np.arange(n_windows)*hop_length + frame_length//2
    ends[ends>sample_size]= sample_size


    for i in range(n_windows):

        x = sample[begs[i]:ends[i]]

        X = fft(x * np.hanning(len(x)))

        
        N = len(x)
        B = math.ceil(N/2)
        df = sr/N

        magnitude = np.abs(X[0:B])
        freq_bins = np.arange(B) * df

        arr[0,i] = np.sum(magnitude * freq_bins)/ np.sum(magnitude)

    return arr  

In [15]:
def spectral_bandwidth(sample, sr= 22050, frame_length= 2048, hop_length=512, p=2):
    
    sample_size= len(sample)
    n_windows = (sample_size)// hop_length + 1
    arr= np.zeros((1, n_windows))


    begs= np.arange(n_windows)*hop_length - frame_length//2
    begs[begs<0]=0
    ends= np.arange(n_windows)*hop_length + frame_length//2
    ends[ends>sample_size]= sample_size

    for i in range(n_windows):

        x = sample[begs[i]:ends[i]]

        X = fft(x * np.hanning(len(x)))

        
        N = len(x)
        B = math.ceil(N/2)
        df = sr/N

        magnitude = np.abs(X[0:B])
        freq_bins = np.arange(B) * df
        centroid = np.sum(magnitude * freq_bins)/ np.sum(magnitude)


        # ! normalization is wrong
        arr[0,i] =  np.sum(magnitude*(freq_bins-centroid)**p)**(1.0/p)


    return arr  

In [16]:
def spectral_flatness(sample, sr= 22050, frame_length= 2048, hop_length=512, safe_value=1e-10):
    
    sample_size= len(sample)
    n_windows = (sample_size)// hop_length + 1
    arr= np.zeros((1, n_windows))


    begs= np.arange(n_windows)*hop_length - frame_length//2
    begs[begs<0]=0
    ends= np.arange(n_windows)*hop_length + frame_length//2
    ends[ends>sample_size]= sample_size

    for i in range(n_windows):

        x = sample[begs[i]:ends[i]]

        X = fft(x * np.hanning(len(x)))

        
        N = len(x)
        B = math.ceil(N/2)
        df = sr/N

        magnitude = np.abs(X[0:B])
        
        magnitude= (magnitude+safe_value) **2
        # magnitude = np.maximum(safe_value, np.abs(X[0:B]) ** 2)
        # arr[0,i] =  (np.prod(magnitude+safe_value)**(1.0/B))/ 

        arr[0,i] = np.exp(np.sum(np.log(magnitude  ))/ B ) / ((np.sum(magnitude)/B)) 




    return arr  

In [17]:
def hertz_to_mel(hertz:float)-> float:
    return 2595 * np.log10(1+hertz/700)

def mel_to_hertz(mel:float) -> float:
    return 700 * (np.power(10, mel/2595)-1)
# now for the implementation of MFCCs

In [18]:
def mfcc(sample: np.ndarray, sr:int, n_mfccs:int=13, frame_length= 2048, hop_length=512):
    sample_size= len(sample)
    n_windows = (sample_size)// hop_length + 1
    begs= np.arange(n_windows)*hop_length - frame_length//2
    begs[begs<0]=0
    ends= np.arange(n_windows)*hop_length + frame_length//2
    ends[ends>sample_size]= sample_size

    delta_f = sr / frame_length
    arr_mfccs = []
    f_max: float = sr/2
    mel_max: float = hertz_to_mel(f_max)
    window_centers = (np.arange(1,41)/41) * mel_max
    window_centers = mel_to_hertz(window_centers)

    # ! meter n_windows
    for i in range(n_windows):
        sample_to_dft: np.ndarray = sample[begs[i]:ends[i]] * np.hanning(ends[i]-begs[i])
        dft_sample = fft(sample_to_dft)
        dft_magnitude = np.abs(dft_sample)[:len(dft_sample)//2+1] # dft coefficients from 0 to 1024 * delta_f
        log_fccs = []
        for j in range(40):
            inf_freq:float = 0
            sup_freq:float = f_max
            mid_freq = window_centers[j]
            if j!=0:
                inf_freq = window_centers[j-1]
            if j<39:     
                sup_freq = window_centers[j+1]
            if delta_f == 0:
                print("lol")
            pos_inf_freq = int(np.ceil(inf_freq / delta_f))
            pos_sup_freq = int(np.floor(sup_freq / delta_f))
            total_cc = 0
            for k in range(pos_inf_freq, min(len(dft_magnitude),pos_sup_freq+1)):
                freq_k = k*delta_f
                coef:float = 0
                if freq_k < mid_freq:
                    if mid_freq - inf_freq ==0:
                        print("lol")
                    perc = (freq_k -inf_freq)/(mid_freq-inf_freq)
                    coef = perc
                else:
                    if sup_freq - mid_freq ==0:
                        print("lol")
                    perc = (freq_k - mid_freq)/(sup_freq-mid_freq)
                    coef = 1-perc
                total_cc += coef * dft_magnitude[k]
            log_total_cc = np.log10(total_cc)
            if np.isnan(log_total_cc) or np.isinf(log_total_cc):
                log_total_cc=0
            log_fccs.append(log_total_cc)
        log_fccs = np.array(log_fccs)
        dft_coefs = dct(log_fccs)[:n_mfccs]
        arr_mfccs.append(dft_coefs)
    ret = np.array(arr_mfccs).T
    # print("ret shape: ",ret.shape)
    # 961413169
    return ret

In [19]:
dirname ='dataset/allSongs'
filename = 'MT0000202045.mp3'
y,fs = librosa.load(dirname+'/'+filename, sr = SR, mono = MONO)

In [20]:

rms1  = librosa.feature.spectral_flatness(y)
print(rms1.shape)
print(rms1)
rms2 = spectral_flatness(y)
print(rms2.shape)
print(rms2)
print()
arr_coef = []
for i in range(len(rms1)):
    arr_coef.append(np.corrcoef(rms1[i], rms2[i]))

# print(arr_coef)
print(np.average(arr_coef))
# print(calculate_stats(rms1))
# print(calculate_stats(rms2))


(1, 1295)
[[0.10003239 0.05057845 0.0463913  ... 0.01123334 0.01457003 0.02427914]]
(1, 1295)
[[0.09080229 0.05019897 0.04687352 ... 0.01134397 0.0139296  0.01996238]]

0.9999843341008465


In [21]:
# print(rms1.tolist())
# print(rms2.tolist())
print()

In [22]:
x= np.arange(3)
aux = np.cumsum(x)>3
aux = 10
aux**(1/2)

3.1622776601683795

In [23]:
def calculate_stats(array:np.ndarray):
    mean= np.mean(array)
    std =  np.std(array)
    skewness = scipy.stats.skew(array)
    kurtosis = scipy.stats.kurtosis(array)
    median = np.median(array)
    maximum = np.max(array)
    minimum = np.min(array)

    return np.array([mean,std,skewness, kurtosis,median, maximum, minimum])


def features_librosa(dirname:str) -> np.ndarray:    
    ans=np.array([])
    i=0
    for filename in os.listdir(dirname):
        print_debug(filename)
        i+=1
        # spectral features
        y,fs = librosa.load(dirname+'/'+filename, sr = SR, mono = MONO)
        mfccs = librosa.feature.mfcc(y, sr=SR, n_mfcc=13)
        spcentroid = librosa.feature.spectral_centroid(y, sr=SR)
        spband = librosa.feature.spectral_bandwidth(y, sr=SR)
        spcontrast = librosa.feature.spectral_contrast(y, sr=SR)
        spflatness = librosa.feature.spectral_flatness(y)
        sprolloff = librosa.feature.spectral_rolloff(y, sr=SR)
        rms = librosa.feature.rms(y)
        zcr = librosa.feature.zero_crossing_rate(y)
        f0 = librosa.yin(y, sr=SR, fmin=20, fmax=11025)
        f0[f0==11025]=0
        all_features_array = np.vstack((mfccs, spcentroid, spband, spcontrast, spflatness, sprolloff, f0, rms, zcr))
        all_stats = np.apply_along_axis(calculate_stats, 1, all_features_array).flatten()


        tempo = librosa.beat.tempo(y,sr=SR)
        aid = np.append(all_stats, tempo)
        # print_debug(aid)
        if i==1:
            ans = np.array(aid)
        else:
            ans= np.vstack((ans,aid))
    # print_debug("ans: ",ans)
    # print_debug("ans shape: ",ans.shape)
    ans = np.array(ans)
    return normalize_features(ans)

In [24]:
#2.3

def features_by_hand(dirname:str) -> np.ndarray:    
    ans=np.array([])
    i=0
    for filename in os.listdir(dirname):
        print_debug(filename)
        i+=1
        # spectral features
        print(i)
        y,fs = librosa.load(dirname+'/'+filename, sr = SR, mono = MONO)
        mfccs = mfcc(y, sr=SR, n_mfccs=13)
        spcentroid = spectral_centroid(y, sr=SR)
        spband = spectral_bandwidth(y, sr=SR)
        spcontrast = librosa.feature.spectral_contrast(y, sr=SR)
        spflatness = spectral_flatness(y)
        sprolloff = spectral_rolloff(y, sr=SR)
        rms_arr = rms(y)
        zcr_arr = zcr(y)
        f0 = librosa.yin(y, sr=SR, fmin=20, fmax=11025)
        f0[f0==11025]=0
        all_features_array = np.vstack((mfccs, spcentroid, spband, spcontrast, spflatness, sprolloff, f0, rms_arr, zcr_arr))
        all_stats = np.apply_along_axis(calculate_stats, 1, all_features_array).flatten()


        tempo = librosa.beat.tempo(y,sr=SR)
        aid = np.append(all_stats, tempo)
        # print_debug(aid)
        if i==1:
            ans = np.array(aid)
        else:
            ans= np.vstack((ans,aid))
    # print_debug("ans: ",ans)
    # print_debug("ans shape: ",ans.shape)
    ans = np.array(ans)
    return normalize_features(ans)

In [25]:
def ex2_2():
    features_norm_obtained = features_librosa('dataset/allSongs')
    export_csv('dataset/song_features.csv', features_norm_obtained)

In [26]:
def ex2_3():
    features_norm_obtained = features_by_hand('dataset/allSongs')
    export_csv('dataset/song_features_by_hand.csv', features_norm_obtained)

In [27]:
# ex2_3()

In [28]:
def euclidean_distance (vec1:np.ndarray, vec2:np.ndarray) -> float:
    return np.linalg.norm(vec1-vec2)

def manhattan_distance(vec1:np.ndarray, vec2:np.ndarray) -> float:
    return np.sum(np.abs(vec1-vec2))

def cosine_distance (vec1:np.ndarray, vec2:np.ndarray) -> float:
    return 1 - np.dot(vec1, vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2))


In [29]:
def distance_matrix(feature_matrix:np.ndarray, distance_function, filename:str):
    lines= len(feature_matrix)
    distance_mat =  np.zeros((lines,lines))

    # distance_mat[np.arange(lines), np.arange(lines)] = distance_function(feature_matrix[np.arange(lines)], feature_matrix[np.arange(lines)])  
    feature_matrix[feature_matrix!=feature_matrix] = 0
    
    for i in range(len(feature_matrix)):
        for j in range(i):
            distance_mat[i,j] = distance_function(feature_matrix[i], feature_matrix[j])
            distance_mat[j,i] = distance_mat[i,j]

    print(filename)
    export_csv(filename, distance_mat)
    return distance_mat

def get_distance_matrices():


    song_features = np.genfromtxt('dataset/song_features.csv', skip_header = 0, delimiter=',') 
    song_features_by_hand = np.genfromtxt('dataset/song_features_by_hand.csv', skip_header = 0, delimiter=',') 
    # the skip header must be set to zero!!!
    top100 = np.genfromtxt('dataset/normalized_features.csv', skip_header = 0, delimiter=',')
    d_functions = [euclidean_distance, manhattan_distance, cosine_distance]
    function_names=['euclidean','manhattan', 'cosine']
    matrices = [ song_features,top100, song_features_by_hand]
    matrix_names=['song_features','top100', 'by_hand']
    n_functions= len(d_functions)
    n_mat = len(matrices)
    n_songs=  len(song_features)
    arr = np.zeros(( n_functions* n_mat,n_songs,n_songs ))
    # arr = [[[] for i in range(len(function_names))] for j in range(len(matrix_names))]
    
    for i in range(len(function_names)):
        for j in range(len(matrices)):
            filename = f"dataset/results/{function_names[i]}_{matrix_names[j]}.csv"
            if not os.path.exists(filename):
                arr[i*n_mat+j]= distance_matrix(matrices[j], d_functions[i],filename)
    # arr = np.array(arr)
    return arr







In [30]:
def read_distance_mats()->np.ndarray:

    arr = np.zeros((NUM_MAT, N_SONGS, N_SONGS ))

    filenames = ['euclidean_song_features','euclidean_top100','euclidean_by_hand','manhattan_song_features','manhattan_top100', 'manhattan_by_hand','cosine_song_features','cosine_top100','cosine_by_hand']

    for i in range(len(filenames)):
        gen =  np.genfromtxt('dataset/results/'+filenames[i]+'.csv', skip_header = 0, delimiter=',')
        arr[i] = gen
    return arr

In [31]:
def get_query_ranking(filename, index, distance_matrices, all_songs)-> np.ndarray:
    print_debug( "Music: ", filename)
    ans = []
    for i in range(len(distance_matrices)):
        line=distance_matrices[i,index]
        sorted_distances = np.argsort(line)
        indices = sorted_distances[1:21]
        ans.append(indices)
        if DEBUG:
            print_debug("According to metric ",i)
            for j in indices:
                print_debug(all_songs[j])
            print_debug("------")
    ans = np.array(ans)
    return ans

def get_rankings(distance_matrices, all_songs, queries)->np.ndarray:
    rks=[]
    for q in queries:
        index = all_songs.index(q)
        rks.append(get_query_ranking ( q,index, distance_matrices, all_songs))
    rks = np.array(rks)
    a,b,c = rks.shape
    ans = np.zeros((b,a,c))
    for i in range(b):
        ans[i] = rks[:,i,:]
    return ans 


In [32]:
def score_distance(md1:np.ndarray, md2:np.ndarray)->int:
    ans=0
    # 1, 3,9,11
    inds = [1,3]
    sps = [9,11]
    for i in inds:
        if md1[i] == md2[i]:
            ans +=1
    # print(ans)
    for s in sps:
        # print_debug("s = ",s, " and md1[s] = ", md1[s], " and md2[s] =",md2[s] )
        aux1= md1[s][1:-1]
        aux2= md2[s][1:-1]
        sp1:list = aux1.split("; ")
        sp2:list = aux2.split("; ")
        # print("sp1 len is ", len(sp1), "and len sp2 is", len(sp2))
        for str_prop in sp1:
            if str_prop in sp2:
                ans +=1
        

    return ans



def get_metadata_ranking(filename:str, index:int, all_songs:list, metadata_matrix:np.ndarray)->np.ndarray:
    # index of the song
    print_debug("Music: ", filename)
    md_file = metadata_matrix[index]
    rating = np.zeros((len(all_songs)))
    for i in range(len(all_songs)):
        md_i = metadata_matrix[i]
        rating[i] = score_distance(md_i, md_file)
        print_debug("Score: ",rating[i])
    rating[index]=-1
    sorted_dists = np.argsort(rating)
    sorted_dists = np.flip(sorted_dists)
    reccoms = sorted_dists[0:20] # the first 20 songs that are supposed to be considered for the ranking
    # print(rating[sorted_dists])
    return reccoms

def get_all_metadata_rankings(all_songs: list, metadata_matrix: np.ndarray, queries:list)->np.ndarray:
    a:list =[]
    for q in queries:
        index:int  = all_songs.index(q)
        a.append(get_metadata_ranking(q,index, all_songs, metadata_matrix))
    a = np.array(a)
    return a

In [33]:
def print_rankings_matrices(ranking_matrix:np.ndarray, all_songs:list, queries:list) -> None :
    for mat in ranking_matrix:
        print_rankings(mat, all_songs, queries)


def print_rankings(ranking_matrix:np.ndarray, all_songs:list, queries:list)->None:
    for i in range(len(queries)):
        q = queries[i]
        print("Recommendation for music", q, ":")
        for r in ranking_matrix[i]:
            print(all_songs[r])
        print("--------")

def get_music_from_pos(music_array, all_songs):
    return [[all_songs[music_array[i,j]] for j in range(len(music_array[i]))] for i in range(len(music_array))]


def save_feature_ranks(all_ranks:np.ndarray, name_array: list, all_songs:list):
    for i in range(len(all_ranks)):
        export_csv(f"dataset/results/ranking_features_metric_{name_array[i]}.csv", get_music_from_pos(all_ranks[i],all_songs), fmt="%s")

In [34]:
def comp_rank(metadata_rank, feature_ranks):
    ret = np.zeros((len(feature_ranks), len(feature_ranks[0])))
    # one number for each type of rating, and for each song
    for k in range(len(feature_ranks)):
        fr = feature_ranks[k] # rating of type k
        for i in range(len(fr)): # for each song in the rating
            f = len([None for j in range(len(fr[i])) if fr[i,j] in metadata_rank[i]]) / len(fr[i])
            print("Precision of rating ", k," and song ", i," is ", f )
            ret[k,i] = f
    return ret


In [35]:
get_distance_matrices()
distance_matrices = read_distance_mats() # reads distance matrices, already obtained
all_songs= os.listdir('dataset/allSongs')
queries = os.listdir('Queries')
distance_names = ['euclidean', 'manhattan', 'cosine']
distance_sources = ['calculated', 'top100', 'calculated_by_hand']
func_names = ['metadata']+ [a+'_'+b for a in distance_names for b in distance_sources]
print(func_names)
all_feature_ranks = get_rankings(distance_matrices, all_songs, queries) # this gets all rankings
metadata_matrix = np.genfromtxt('dataset/panda_dataset_taffc_metadata.csv', delimiter=',',skip_header = 1, encoding = None, dtype=None)
metadata_rankings = get_all_metadata_rankings(all_songs, metadata_matrix, queries) # getting rankings based on metadata
metadata_rankings = metadata_rankings[np.newaxis, :,:]
all_rankings = np.vstack((metadata_rankings, all_feature_ranks)).astype(np.int32)
print_rankings_matrices(all_rankings, all_songs, queries)
save_feature_ranks(all_rankings, func_names, all_songs) # the rankings are now saved in the disk
cr = comp_rank(all_rankings[0], all_rankings[1:])
print(cr.shape)
export_csv("dataset/results/rating_eval.csv", cr, '%f')




dataset/results/euclidean_by_hand.csv
dataset/results/manhattan_by_hand.csv
dataset/results/cosine_by_hand.csv


IndexError: index 6 is out of bounds for axis 0 with size 6